# Exploring free responses
This notebook provide sample [EDSL](https://docs.expectedparrot.com/) code demonstrating ways of modifying instructions for free response questions.

EDSL is an open-source library for simulating surveys and experiments with AI. Please see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started.

## Creating free response questions

In [1]:
from edsl import QuestionFreeText

q1 = QuestionFreeText(
    question_name="pasttime",
    question_text="What is your favorite pasttime? {{ instruction }}",
)

q2 = QuestionFreeText(
    question_name="vacation",
    question_text="What is your favorite vacation destination? {{ instruction }}",
)

## Parameterizing the questions with special instructions

In [2]:
from edsl import ScenarioList, Scenario

special_instructions = [
    "Be as specific as possible.",
    "Be concise!",
    "Wax poetic here.",
]

scenarios = ScenarioList(
    Scenario({"instruction": i}) for i in special_instructions
)

## Creating agent personas with specific survey contexts

In [3]:
from edsl import AgentList, Agent

personas = [
    "You are middle-aged.",
    "You are a senior citizen.",
    "You are a young adult.",
]

instructions = [
    "You are handwriting answers in a paper survey.",
    "You are typing answers in an online survey.",
    "You are providing answers verbally to a researcher in a live interview.",
]

agents = AgentList(
    Agent(traits={"persona": p}, instruction=i) for p in personas for i in instructions
)

## Selecting LLMs

In [4]:
from edsl import ModelList, Model

models = ModelList(
    Model(m) for m in ["gpt-4o", "gemini-pro"]
)

## Administering the survey

In [5]:
from edsl import Survey

survey = Survey(questions = [q1, q2])

results = survey.by(scenarios).by(agents).by(models).run()

In [6]:
(
    results.sort_by("model", "persona", "agent_instruction")
    .select("model", "persona", "agent_instruction", "scenario.*", "answer.*")
    .print(format="rich")
)

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ model      ┃ agent             ┃ agent             ┃ scenario          ┃ answer            ┃ answer             ┃
┃ .model     ┃ .persona          ┃ .agent_instructi… ┃ .instruction      ┃ .pasttime         ┃ .vacation          ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ gemini-pro │ You are a senior  │ You are           │ Be as specific as │ My favorite       │ My favorite        │
│            │ citizen.          │ handwriting       │ possible.         │ pastime is        │ vacation           │
│            │                   │ answers in a      │                   │ tending to my     │ destination is the │
│            │                   │ paper survey.     │                   │ rose garden. I    │ town of Branson,   │
│            │                   │                   │                   │ have been growing │ Missouri. I enjoy  │
│            │                   │                   │                   │ roses for over 50 │ the live music     │
│            │                   │                   │                   │ years, and I have │ shows, the         │
│            │                   │                   │                   │ a collection of   │ beautiful scenery  │
│            │                   │                   │                   │ over 100          │ of the Ozark       │
│            │                   │                   │                   │ different         │ Mountains, and the │
│            │                   │                   │                   │ varieties. I love │ friendly people. I │
│            │                   │                   │                   │ the challenge of  │ especially enjoy   │
│            │                   │                   │                   │ growing new       │ the shows at the   │
│            │                   │                   │                   │ roses, and I am   │ Sight & Sound      │
│            │                   │                   │                   │ always            │ Theatres, which    │
│            │                   │                   │                   │ experimenting     │ are known for      │
│            │                   │                   │                   │ with different    │ their spectacular  │
│            │                   │                   │                   │ techniques to     │ sets and costumes. │
│            │                   │                   │                   │ improve my        │ I also like to     │
│            │                   │                   │                   │ blooms. I also    │ visit the Titanic  │
│            │                   │                   │                   │ enjoy sharing my  │ Museum Attraction, │
│            │                   │                   │                   │ roses with        │ which has a        │
│            │                   │                   │                   │ others, and I     │ replica of the     │
│            │                   │                   │                   │ often give        │ Titanic ship and   │
│            │                   │                   │                   │ cuttings to       │ artifacts from the │
│            │                   │                   │                   │ friends and       │ actual ship.       │
│            │                   │                   │                   │ neighbors.        │                    │
├────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼────────────────────┤
│ gemini-pro │ You are a senior  │ You are           │ Be concise!       │ Reading           │ Hawaii             │
│            │ citizen.          │ handwriting       │                   │                   │                    │
│            │                   │ answers in a      │  

## Posting to the Coop

In [7]:
survey.push(description = "Free text: example survey")

{'description': 'Free text: example survey',
 'object_type': 'survey',
 'url': 'https://www.expectedparrot.com/content/1da7b60e-c106-4346-a8f8-691ffe528ec3',
 'uuid': '1da7b60e-c106-4346-a8f8-691ffe528ec3',
 'version': '0.1.33.dev1',
 'visibility': 'unlisted'}

In [8]:
scenarios.push(description = "Free text: example scenarios")

{'description': 'Free text: example scenarios',
 'object_type': 'scenario_list',
 'url': 'https://www.expectedparrot.com/content/793987a8-e5df-4c02-b704-11a1c0db8b89',
 'uuid': '793987a8-e5df-4c02-b704-11a1c0db8b89',
 'version': '0.1.33.dev1',
 'visibility': 'unlisted'}

In [9]:
agents.push(description = "Free text: example agents")

{'description': 'Free text: example agents',
 'object_type': 'agent_list',
 'url': 'https://www.expectedparrot.com/content/5d0b4cc3-34ec-4494-b32f-550f21c3719c',
 'uuid': '5d0b4cc3-34ec-4494-b32f-550f21c3719c',
 'version': '0.1.33.dev1',
 'visibility': 'unlisted'}

In [10]:
results.push(description = "Free text: example survey results")

{'description': 'Free text: example survey results',
 'object_type': 'results',
 'url': 'https://www.expectedparrot.com/content/f1b0e455-8b2f-4274-bcff-6fbc68e92520',
 'uuid': 'f1b0e455-8b2f-4274-bcff-6fbc68e92520',
 'version': '0.1.33.dev1',
 'visibility': 'unlisted'}

In [12]:
from edsl import Notebook

In [13]:
n = Notebook(path = "free_responses.ipynb")

In [14]:
n.push(description = "Example code exploring free text instructions", visibility = "public")

{'description': 'Example code exploring free text instructions',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/044d7fe9-2e6f-446b-831a-34039ac18623',
 'uuid': '044d7fe9-2e6f-446b-831a-34039ac18623',
 'version': '0.1.33.dev1',
 'visibility': 'public'}